In [1]:
pip install gradio


In [9]:
import pandas as pd
import gradio as gr
import os
import platform
from requests import get
import joblib

# Define the CSV file to store the data
output_file = "survey_results.csv"

# Define the column names for the blank CSV initialization
columns_order = [
    *[f"Q{i}A" for i in range(1, 43)],
    "country", "source",
    *[f"TIPI{i}" for i in range(1, 11)],
    *[f"VCL{i}" for i in range(1, 17)],
    "education", "urban", "gender", "engnat", "age", "screensize",
    "uniquenetworklocation", "hand", "religion", "orientation", "race",
    "voted", "married", "familysize", "major"
]

# Create a blank CSV file if it does not exist
if not os.path.exists(output_file):
    # Create an empty DataFrame with the required columns
    empty_df = pd.DataFrame(columns=columns_order)
    empty_df.to_csv(output_file, index=False)
    print(f"Blank CSV file '{output_file}' created.")
else:
    print(f"File '{output_file}' already exists.")

# DASS Questions and Options
dass_questions = [
    f"Q{i}A: {desc}"
    for i, desc in enumerate(
        [
            "I found myself getting upset by quite trivial things.",
            "I was aware of dryness of my mouth.",
            "I couldn't seem to experience any positive feeling at all.",
            "I experienced breathing difficulty (e.g., excessively rapid breathing).",
            "I just couldn't seem to get going.",
            "I tended to over-react to situations.",
            "I had a feeling of shakiness.",
            "I found it difficult to relax.",
            "I found myself in situations that made me so anxious I was most relieved when they ended.",
            "I felt that I had nothing to look forward to.",
            "I found myself getting upset rather easily.",
            "I felt that I was using a lot of nervous energy.",
            "I felt sad and depressed.",
            "I found myself getting impatient when I was delayed.",
            "I had a feeling of faintness.",
            "I felt that I had lost interest in just about everything.",
            "I felt I wasn't worth much as a person.",
            "I felt that I was rather touchy.",
            "I perspired noticeably in the absence of high temperatures or physical exertion.",
            "I felt scared without any good reason.",
            "I felt that life wasn't worthwhile.",
            "I found it hard to wind down.",
            "I had difficulty in swallowing.",
            "I couldn't seem to get any enjoyment out of the things I did.",
            "I was aware of the action of my heart in the absence of physical exertion.",
            "I felt down-hearted and blue.",
            "I found that I was very irritable.",
            "I felt I was close to panic.",
            "I found it hard to calm down after something upset me.",
            "I feared that I would be 'thrown' by some trivial but unfamiliar task.",
            "I was unable to become enthusiastic about anything.",
            "I found it difficult to tolerate interruptions to what I was doing.",
            "I was in a state of nervous tension.",
            "I felt I was pretty worthless.",
            "I was intolerant of anything that kept me from getting on with what I was doing.",
            "I felt terrified.",
            "I could see nothing in the future to be hopeful about.",
            "I felt that life was meaningless.",
            "I found myself getting agitated.",
            "I was worried about situations in which I might panic and make a fool of myself.",
            "I experienced trembling.",
            "I found it difficult to work up the initiative to do things.",
        ],
        start=1,
    )
]
dass_options = {
    "Did not apply to me at all": 1,
    "Applied to me to some degree, or some of the time": 2,
    "Applied to me to a considerable degree, or a good part of the time": 3,
    "Applied to me very much, or most of the time": 4,
}

# TIPI Questions
tipi_questions = [
    f"TIPI{i}: {desc}"
    for i, desc in enumerate(
        [
            "Extraverted, enthusiastic.",
            "Critical, quarrelsome.",
            "Dependable, self-disciplined.",
            "Anxious, easily upset.",
            "Open to new experiences, complex.",
            "Reserved, quiet.",
            "Sympathetic, warm.",
            "Disorganized, careless.",
            "Calm, emotionally stable.",
            "Conventional, uncreative.",
        ],
        start=1,
    )
]
tipi_options = {
    "Disagree strongly": 1,
    "Disagree moderately": 2,
    "Disagree a little": 3,
    "Neither agree nor disagree": 4,
    "Agree a little": 5,
    "Agree moderately": 6,
    "Agree strongly": 7,
}

# VCL Words
vcl_words = [f"VCL{i}: {word}" for i, word in enumerate(
    [
        "boat", "incoherent", "pallid", "robot", "audible", "cuivocal",
        "paucity", "epistemology", "florted", "decide", "pastiche",
        "verdid", "abysmal", "lucid", "betray", "funny"
    ],
    start=1,
)]

# Demographics and metadata
demographics = {
    "education": {"Less than high school": 1, "High school": 2, "University degree": 3, "Graduate degree": 4},
    "urban": {"Rural": 1, "Suburban": 2, "Urban": 3},
    "gender": {"Male": 1, "Female": 2, "Other": 3},
    "engnat": {"Yes": 1, "No": 2},
    "age": None,  # Numeric input
    "hand": {"Right": 1, "Left": 2, "Both": 3},
    "religion": {
        "Agnostic": 1, "Atheist": 2, "Buddhist": 3, "Christian (Catholic)": 4,
        "Christian (Mormon)": 5, "Christian (Protestant)": 6, "Christian (Other)": 7,
        "Hindu": 8, "Jewish": 9, "Muslim": 10, "Sikh": 11, "Other": 12
    },
    "orientation": {"Heterosexual": 1, "Bisexual": 2, "Homosexual": 3, "Asexual": 4, "Other": 5},
    "race": {"Asian": 10, "Arab": 20, "Black": 30, "Indigenous Australian": 40, "Native American": 50, "White": 60, "Other": 70},
    "voted": {"Yes": 1, "No": 2},
    "married": {"Never married": 1, "Currently married": 2, "Previously married": 3},
    "familysize": None,  # Numeric input
    "major": None,       # Text input
}

# Derived fields
def derive_information():
    try:
        ip_data = get("https://ipinfo.io").json()
        country = ip_data.get("country", "Unknown")
    except Exception:
        country = "Unknown"
    screensize = 2 if platform.system() in ["Linux", "Windows"] else 1
    uniquenetworklocation = 1
    source = 1
    return country, screensize, uniquenetworklocation, source


# Save user inputs to the CSV file
def save_input_to_csv(*inputs):
    """
    Collects user responses and appends them to the CSV file.
    """
    try:
        # Extract terms agreement
        terms_agreement = inputs[-1]  # Last input is the agreement checkbox
        if not terms_agreement:
            return "You must agree to the terms and conditions to submit your responses."

        # Derive metadata
        country, screensize, uniquenetworklocation, source = derive_information()

        # Process survey responses
        dass_responses = [dass_options.get(inputs[i], None) for i in range(len(dass_questions))]
        tipi_responses = [tipi_options.get(inputs[len(dass_questions) + i], None) for i in range(len(tipi_questions))]
        vcl_responses = [1 if inputs[len(dass_questions) + len(tipi_questions) + i] else 0 for i in range(len(vcl_words))]

        # Extract demographics
        demographics_responses = {
            key: (
                demographics[key].get(inputs[len(dass_questions) + len(tipi_questions) + len(vcl_words) + idx], None)
                if key not in ["age", "familysize", "major"] else inputs[len(dass_questions) + len(tipi_questions) + len(vcl_words) + idx]
            )
            for idx, key in enumerate(demographics.keys())
        }

        # Prepare data
        data = {
            **{f"Q{i}A": dass_responses[i - 1] for i in range(1, 43)},
            "country": country,
            "source": source,
            **{f"TIPI{i}": tipi_responses[i - 1] for i in range(1, 11)},
            **{f"VCL{i}": vcl_responses[i - 1] for i in range(1, 17)},
            **demographics_responses,
            "screensize": screensize,
            "uniquenetworklocation": uniquenetworklocation,
        }

        # Convert to DataFrame
        df = pd.DataFrame([data])

        # Append to the CSV file
        df.to_csv(output_file, mode="a", header=False, index=False)
        return "Inputs saved successfully!"

    except Exception as e:
        # Return a detailed error message
        print(f"Error: {e}")
        return f"An error occurred while saving the inputs: {e}"


Blank CSV file 'survey_results.csv' created.


In [10]:
import pandas as pd
data=pd.read_csv('survey_results.csv')
data.head()


,Q1A,Q2A,Q3A,Q4A,Q5A,Q6A,Q7A,Q8A,Q9A,Q10A,...,screensize,uniquenetworklocation,hand,religion,orientation,race,voted,married,familysize,major


In [11]:
import numpy as np
data_1=data.copy()
data_1['major']=data_1['major'].replace(np.nan,'No Degree')
#time = [i for i in data_1.iloc[:,0:126] if  'E' in i]
#position = [i for i in data_1.iloc[:,0:126] if  'I' in i]
#data_1=data_1.drop(position,axis=1)
#data_1=data_1.drop(time,axis=1)
data_1=data_1.drop(data_1.iloc[:,43:44],axis=1)
data_2=data_1.copy()
data_2=data_2.drop(data_2.iloc[:,53:69],axis=1)
data_2=data_2.replace(to_replace=0,value=3)
data_2=data_2.rename(columns={'TIPI1':'Extraverted-enthusiastic','TIPI2':'Critical-quarrelsome',
                            'TIPI3':'Dependable-self_disciplined','TIPI4':'Anxious-easily upset',
                            'TIPI5':'Open to new experiences-complex','TIPI6':'Reserved-quiet',
                            'TIPI7':'Sympathetic-warm','TIPI8':'Disorganized-careless','TIPI9':'Calm-emotionally_stable',
                            'TIPI10':'Conventional-uncreative'})
print('Shape',data_2.shape)
print('Attributes',data_2.columns)

Shape (0, 68)
Attributes Index(['Q1A', 'Q2A', 'Q3A', 'Q4A', 'Q5A', 'Q6A', 'Q7A', 'Q8A', 'Q9A', 'Q10A',
       'Q11A', 'Q12A', 'Q13A', 'Q14A', 'Q15A', 'Q16A', 'Q17A', 'Q18A', 'Q19A',
       'Q20A', 'Q21A', 'Q22A', 'Q23A', 'Q24A', 'Q25A', 'Q26A', 'Q27A', 'Q28A',
       'Q29A', 'Q30A', 'Q31A', 'Q32A', 'Q33A', 'Q34A', 'Q35A', 'Q36A', 'Q37A',
       'Q38A', 'Q39A', 'Q40A', 'Q41A', 'Q42A', 'country',
       'Extraverted-enthusiastic', 'Critical-quarrelsome',
       'Dependable-self_disciplined', 'Anxious-easily upset',
       'Open to new experiences-complex', 'Reserved-quiet', 'Sympathetic-warm',
       'Disorganized-careless', 'Calm-emotionally_stable',
       'Conventional-uncreative', 'education', 'urban', 'gender', 'engnat',
       'age', 'screensize', 'uniquenetworklocation', 'hand', 'religion',
       'orientation', 'race', 'voted', 'married', 'familysize', 'major'],
      dtype='object')


In [12]:
data_2=data_2.replace([np.inf, -np.inf], np.nan)
data_2=data_2.dropna()

In [13]:
def condition(x):
    if x<=10:
        return 'Under 10'
    if  10<=x<=16:
        return ' Primary Children'
    if 17<=x<=21:
        return 'Secondary Children'
    if 21<=x<=35:
        return 'Adults'
    if 36<=x<=48:
        return 'Elder Adults'
    if x>=49:
        return 'Older People'

data_2['Age_Groups']=data_2['age'].apply(condition)
data_2.head()

,Q1A,Q2A,Q3A,Q4A,Q5A,Q6A,Q7A,Q8A,Q9A,Q10A,...,uniquenetworklocation,hand,religion,orientation,race,voted,married,familysize,major,Age_Groups


In [14]:
new_data=data_2.iloc[:,42:]
data_3 = data_2.filter(regex=r'Q\d{1,2}A')
data_3.head()

,Q1A,Q2A,Q3A,Q4A,Q5A,Q6A,Q7A,Q8A,Q9A,Q10A,...,Q33A,Q34A,Q35A,Q36A,Q37A,Q38A,Q39A,Q40A,Q41A,Q42A


In [15]:
def sub(data_3):
    return data_3.subtract(1,axis=1)
data_3=sub(data_3)
DASS_keys = {'Depression': [3, 5, 10, 13, 16, 17, 21, 24, 26, 31, 34, 37, 38, 42],
             'Anxiety': [2, 4, 7, 9, 15, 19, 20, 23, 25, 28, 30, 36, 40, 41],
             'Stress': [1, 6, 8, 11, 12, 14, 18, 22, 27, 29, 32, 33, 35, 39]}
Dep = []
for i in DASS_keys["Depression"]:
    Dep.append('Q'+str(i)+'A')
Stress = []
for i in DASS_keys["Stress"]:
    Stress.append('Q'+str(i)+'A')
Anx = []
for i in DASS_keys["Anxiety"]:
    Anx.append('Q'+str(i)+'A')
depression= data_3.filter(Dep)
stress = data_3.filter(Stress)
anxiety = data_3.filter(Anx)

In [16]:
def scores(source):
    col=list(source)
    source['Total_Count']=source[col].sum(axis=1)
    return source
depression=scores(depression)
stress=scores(stress)
anxiety=scores(anxiety)

In [17]:
Depression=pd.merge(depression,new_data,how='left',left_index=True,right_index=True)
Depression.head()

,Q3A,Q5A,Q10A,Q13A,Q16A,Q17A,Q21A,Q24A,Q26A,Q31A,...,uniquenetworklocation,hand,religion,orientation,race,voted,married,familysize,major,Age_Groups


In [18]:
Stress=pd.merge(stress,new_data,how='inner',left_index=True,right_index=True)
Stress.head()

,Q1A,Q6A,Q8A,Q11A,Q12A,Q14A,Q18A,Q22A,Q27A,Q29A,...,uniquenetworklocation,hand,religion,orientation,race,voted,married,familysize,major,Age_Groups


In [19]:
Anxiety=pd.merge(anxiety,new_data,how='inner',left_index=True,right_index=True)
Anxiety.head()

,Q2A,Q4A,Q7A,Q9A,Q15A,Q19A,Q20A,Q23A,Q25A,Q28A,...,uniquenetworklocation,hand,religion,orientation,race,voted,married,familysize,major,Age_Groups


In [20]:
def condition(x):
    if x<=9:
        return 'Normal'
    if  10<=x<=13:
        return 'Mild'
    if 14<=x<=20:
        return 'Moderate'
    if 21<=x<=27:
        return 'Severe'
    if x>28:
        return 'Extremely Severe'

Depression['Condition']=Depression['Total_Count'].apply(condition)
Depression.head()




,Q3A,Q5A,Q10A,Q13A,Q16A,Q17A,Q21A,Q24A,Q26A,Q31A,...,hand,religion,orientation,race,voted,married,familysize,major,Age_Groups,Condition


In [21]:
def condition(x):
    if x<=14:
        return 'Normal'
    if  15<=x<=18:
        return 'Mild'
    if 19<=x<=25:
        return 'Moderate'
    if 26<=x<=33:
        return 'Severe'
    if x>=34:
        return 'Extremely Severe'

Stress['Condition']=Stress['Total_Count'].apply(condition)

Stress.head()

,Q1A,Q6A,Q8A,Q11A,Q12A,Q14A,Q18A,Q22A,Q27A,Q29A,...,hand,religion,orientation,race,voted,married,familysize,major,Age_Groups,Condition


In [22]:
def condition(x):
    if x<=7:
        return 'Normal'
    if  8<=x<=9:
        return 'Mild'
    if 10<=x<=14:
        return 'Moderate'
    if 15<=x<=19:
        return 'Severe'
    if x>19:
        return 'Extremely Severe'

Anxiety['Condition']=Anxiety['Total_Count'].apply(condition)
Anxiety.head()

,Q2A,Q4A,Q7A,Q9A,Q15A,Q19A,Q20A,Q23A,Q25A,Q28A,...,hand,religion,orientation,race,voted,married,familysize,major,Age_Groups,Condition


In [23]:
def change_var(x):
    if x=='Primary Children':
        return 0
    elif x=='Secondary Children':
        return 1
    elif x=='Adults':
        return 2
    elif x=='Elder Adults':
        return 3
    elif x=='Older People':
        return 4



Depression['Age_Groups']=Depression['Age_Groups'].apply(change_var)
Stress['Age_Groups']=Stress['Age_Groups'].apply(change_var)
Anxiety['Age_Groups']=Anxiety['Age_Groups'].apply(change_var)

In [24]:
Depression=Depression.dropna()
Stress=Stress.dropna()
Anxiety=Anxiety.dropna()

In [25]:
Depression=Depression.drop(columns=['Total_Count','country','age','major','Condition'])
Stress=Stress.drop(columns=['Total_Count','country','age','major','Condition'])
Anxiety=Anxiety.drop(columns=['Total_Count','country','age','major','Condition'])

In [26]:
print(Depression.columns)
print(Stress.columns)
print(Anxiety.columns)


Index(['Q3A', 'Q5A', 'Q10A', 'Q13A', 'Q16A', 'Q17A', 'Q21A', 'Q24A', 'Q26A',
       'Q31A', 'Q34A', 'Q37A', 'Q38A', 'Q42A', 'Extraverted-enthusiastic',
       'Critical-quarrelsome', 'Dependable-self_disciplined',
       'Anxious-easily upset', 'Open to new experiences-complex',
       'Reserved-quiet', 'Sympathetic-warm', 'Disorganized-careless',
       'Calm-emotionally_stable', 'Conventional-uncreative', 'education',
       'urban', 'gender', 'engnat', 'screensize', 'uniquenetworklocation',
       'hand', 'religion', 'orientation', 'race', 'voted', 'married',
       'familysize', 'Age_Groups'],
      dtype='object')
Index(['Q1A', 'Q6A', 'Q8A', 'Q11A', 'Q12A', 'Q14A', 'Q18A', 'Q22A', 'Q27A',
       'Q29A', 'Q32A', 'Q33A', 'Q35A', 'Q39A', 'Extraverted-enthusiastic',
       'Critical-quarrelsome', 'Dependable-self_disciplined',
       'Anxious-easily upset', 'Open to new experiences-complex',
       'Reserved-quiet', 'Sympathetic-warm', 'Disorganized-careless',
       'Calm-emotionally

In [30]:
import joblib
import gradio as gr
import pandas as pd
import os

# Define the output file name
output_file = "survey_results.csv"

# Prediction function
def predict_with_inputs(*inputs):
    """
    Save user inputs to CSV, display them, and run predictions for Depression, Stress, and Anxiety.
    """
    try:
        # Display user inputs (as a confirmation to the user)
        user_responses = {
            "Input DASS Responses": inputs[:len(dass_questions)],
            "Input TIPI Responses": inputs[len(dass_questions): len(dass_questions) + len(tipi_questions)],
            "Input VCL Responses": inputs[len(dass_questions) + len(tipi_questions): len(dass_questions) + len(tipi_questions) + len(vcl_words)],
            "Demographic Responses": inputs[len(dass_questions) + len(tipi_questions) + len(vcl_words):-1],
            "Agreed to Terms": inputs[-1]
        }

        # Check if terms and conditions are agreed
        if not inputs[-1]:  # The last input is the terms agreement checkbox
            return (
                "You must agree to the terms and conditions to submit your responses.",
                None,
                None,
                None,
            )

        # Prepare data for saving
        dass_responses = inputs[:len(dass_questions)]
        tipi_responses = inputs[len(dass_questions): len(dass_questions) + len(tipi_questions)]
        vcl_responses = [1 if x else 0 for x in inputs[len(dass_questions) + len(tipi_questions): len(dass_questions) + len(tipi_questions) + len(vcl_words)]]
        demographics_responses = inputs[len(dass_questions) + len(tipi_questions) + len(vcl_words):-1]

        # Combine all responses into a single row
        row_data = list(dass_responses) + list(tipi_responses) + list(vcl_responses) + list(demographics_responses)

        # Load existing CSV or create a new one
        if not os.path.exists(output_file):
            columns = [
                *[f"Q{i}A" for i in range(1, 43)],
                *[f"TIPI{i}" for i in range(1, 11)],
                *[f"VCL{i}" for i in range(1, 17)],
                *demographics.keys()
            ]
            df = pd.DataFrame([row_data], columns=columns)
            df.to_csv(output_file, index=False)
        else:
            # Append new data to the existing file
            df = pd.DataFrame([row_data])
            df.to_csv(output_file, mode="a", header=False, index=False)

        # Load pre-trained models
        depression_model = joblib.load("random_forest_Depression.pkl")
        stress_model = joblib.load("random_forest_stress.pkl")
        anxiety_model = joblib.load("random_forest_anxiety.pkl")

        # Make predictions
        depression_prediction = depression_model.predict(Depression)[-1]  # Predict for the latest entry
        stress_prediction = stress_model.predict(Stress)[-1]
        anxiety_prediction = anxiety_model.predict(Anxiety)[-1]

        # Return user inputs and predictions
        return (
            "Responses saved successfully!",
            f"Depression Level: {depression_prediction}",
            f"Stress Level: {stress_prediction}",
            f"Anxiety Level: {anxiety_prediction}",
        )
    except Exception as e:
        return (
            f"An error occurred during prediction: {e}",
            None,
            None,
            None,
        )


# Gradio Inputs
dass_inputs = [gr.Radio(label=q, choices=list(dass_options.keys())) for q in dass_questions]
tipi_inputs = [gr.Radio(label=q, choices=list(tipi_options.keys())) for q in tipi_questions]
vcl_inputs = [gr.Checkbox(label=q) for q in vcl_words]

# Demographic inputs
demographics_inputs = [
    gr.Radio(label=key.capitalize(), choices=list(options.keys())) if options else
    (gr.Number(label=key.capitalize()) if key in ["age", "familysize"] else gr.Textbox(label="Major"))
    for key, options in demographics.items()
]

# Terms and Conditions input
terms_and_conditions = gr.Checkbox(label="I agree to the terms and conditions", value=False)

# Combine all inputs
all_inputs = dass_inputs + tipi_inputs + vcl_inputs + demographics_inputs + [terms_and_conditions]

# Gradio Interface
gr.Interface(
    fn=predict_with_inputs,
    inputs=all_inputs,
    outputs=[
        "text",    # Confirmation message
        "text",    # Depression prediction
        "text",    # Stress prediction
        "text"     # Anxiety prediction
    ],
    title="Survey with Predictions",
    description="Complete the survey, agree to terms and conditions, and get predictions for Depression, Stress, and Anxiety.",
).launch()


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
